# I- 🧾 Description Générale du Projet

Ce projet de machine learning vise à développer un système capable de classer automatiquement les tickets de support client selon leur niveau de priorité (faible, moyen, élevé), à partir de données disponibles dès leur création. L’objectif est de faciliter la gestion des demandes techniques en permettant aux équipes de support de se concentrer rapidement sur les tickets les plus critiques.

Le dataset utilisé est entièrement synthétique et contient 50 000 tickets de support provenant de 25 entreprises fictives. Il a été conçu pour simuler des situations réalistes avec un mélange de données numériques, catégorielles et textuelles simplifiées.

Le projet combine deux approches complémentaires :

**Apprentissage supervisé** : pour entraîner des modèles de classification (ex. : Random Forest, XGBoost) à prédire la priorité des tickets.

**Apprentissage non supervisé** : pour explorer les données via des techniques de regroupement (clustering) et détecter des structures cachées.

Ce projet est une excellente opportunité pédagogique pour mettre en pratique l’analyse exploratoire, le prétraitement des données, la modélisation, l’évaluation de performances, et l’interprétation des résultats dans un contexte métier réaliste.

# II- 🧠 Business Understanding

## 1- 📊 Business Objectives (BO) vs Data Science Objectives (DSO)

| **Business Objective (BO)** | **Data Science Objective (DSO)** |
|------------------------------|----------------------------------|
| **1. Améliorer la gestion des tickets** : Automatiser l’attribution d’une priorité (faible, moyenne, élevée) dès la création d’un ticket. | Construire un **modèle de classification supervisée** (Random Forest, XGBoost, etc.) pour prédire la variable `priority` avec une performance > 80% (mesurée par F1-score ou accuracy). |
| **2. Optimiser l’utilisation des ressources humaines** : Identifier rapidement les tickets critiques pour mieux répartir les tâches. | Mettre en place un **système de scoring basé sur les prédictions ML** afin de classer automatiquement les tickets par ordre de criticité. |
| **3. Améliorer la satisfaction client** : Réduire le temps de réponse sur les tickets critiques. | Analyser l’impact de variables comme `downtime_min`, `error_rate_pct` ou `customer_sentiment` et intégrer ces signaux dans le modèle pour améliorer la détection des cas urgents. |
| **4. Analyse stratégique** : Comprendre l’influence de facteurs comme la taille d’entreprise, la région ou le produit sur la priorité des tickets. | Réaliser une **analyse exploratoire (EDA)** et utiliser des méthodes d’explicabilité (Feature Importance, SHAP) pour identifier les variables qui influencent le plus la priorité. |
| **5. Innovation et automatisation** : Explorer de nouvelles approches pour améliorer la gestion des tickets. | - **Supervisé** : Comparer plusieurs modèles ML (Logistic Regression, Random Forest, XGBoost, etc.) et choisir le meilleur.<br>- **Non supervisé** : Appliquer du **clustering (K-Means, DBSCAN, PCA)** pour regrouper les tickets similaires et détecter des anomalies. |


## 2- ❓ Problématique Générale

Les entreprises reçoivent un grand volume de tickets de support qu’il est difficile de prioriser manuellement.  
Comment développer un modèle de Machine Learning capable de prédire automatiquement la priorité des tickets et, en parallèle, utiliser des méthodes de clustering pour regrouper les tickets similaires et détecter des anomalies ?


# II- 🧠 Data Understanding

## 1- 📂 Importation de Dataset (Excel)

Nous allons charger le dataset au format **CSV** en utilisant la librairie **pandas**.  
La fonction `pd.read_csv()` permet de lire directement un fichier `.csv`. 

In [2]:
# Importer pandas
import pandas as pd

# Charger le fichier CSV
# Remplace "Support_tickets.csv" par le chemin exact de ton fichier
df = pd.read_csv("Support_tickets.csv")

# Vérifier les dimensions du dataset
print("Shape :", df.shape)

# Aperçu des 5 premières lignes
df.head()


Shape : (50000, 33)


,ticket_id,day_of_week,day_of_week_num,company_id,company_size,company_size_cat,industry,industry_cat,customer_tier,customer_tier_cat,...,downtime_min,payment_impact_flag,security_incident_flag,data_loss_flag,has_runbook,customer_sentiment,customer_sentiment_cat,description_length,priority,priority_cat
0,1000000000,Wed,3,100015,Small,1,media,7,Basic,1,...,6,0,0,0,0,neutral,2,227,low,1
1,1000000001,Sat,6,100023,Small,1,healthcare,5,Basic,1,...,2,0,0,0,0,neutral,2,461,low,1
2,1000000002,Mon,1,100012,Small,1,gaming,4,Basic,1,...,0,0,0,0,1,positive,3,306,low,1
3,1000000003,Wed,3,100003,Small,1,media,7,Plus,2,...,16,0,0,0,1,neutral,2,363,medium,2
4,1000000004,Mon,1,100019,Small,1,ecommerce,2,Plus,2,...,6,0,0,0,0,neutral,2,442,low,1


## 2- 🗂️ Schéma des Données

Le dataset contient les colonnes suivantes :

#### Identifiants & Temps
- **ticket_id** : Identifiant unique du ticket (int64)
- **day_of_week** : Jour de la semaine (Lun–Dim)
- **day_of_week_num** : Numéro du jour (1–7 ; Lun=1)

#### Profil de l’Entreprise (répliqué sur chaque ligne)
- **company_id** : Identifiant de l’entreprise (int)
- **company_size** : Taille de l’entreprise (Petite/Moyenne/Grande)
- **industry** : Secteur d’activité (7 catégories)
- **customer_tier** : Niveau de service client (Basic/Plus/Enterprise)
- **org_users** : Nombre d’utilisateurs actifs (jusqu’à ~10 000 pour les grandes entreprises)

#### Contexte
- **region** : Région (AMER/EMEA/APAC)
- **past_30d_tickets** : Nombre de tickets sur les 30 derniers jours (int)
- **past_90d_incidents** : Nombre d’incidents sur les 90 derniers jours (int)

#### Produit & Canal
- **product_area** : Domaine produit (authentification, facturation, mobile, pipeline de données, analytique, notifications)
- **booking_channel** : Canal de création (web, email, chat, téléphone)
- **reported_by_role** : Rôle du déclarant (support, devops, chef produit, finance, direction)

#### Impact & Indicateurs
- **customers_affected** : Nombre de clients impactés (int, distribution asymétrique)
- **error_rate_pct** : Taux d’erreur (%) (float, 0–100 ; parfois 0.0 si non mesuré)
- **downtime_min** : Durée d’indisponibilité (minutes, 0 si seulement dégradé)
- **payment_impact_flag** : Impact sur paiement (0/1)
- **security_incident_flag** : Incident de sécurité (0/1)
- **data_loss_flag** : Perte de données (0/1)
- **has_runbook** : Procédure disponible (0/1)

#### Texte & Proxies
- **customer_sentiment** : Sentiment client (négatif/neutre/positif ; 0 = manquant)
- **description_length** : Longueur de la description (int, 20–2000)

#### Cible
- **priority** : Priorité du ticket (faible/moyenne/élevée)
- **priority_cat** : Priorité (1/2/3)

**Remarques :**
- Jeu de données entièrement synthétique, adapté à l’éducation et au benchmarking.
- Pas d’ordre temporel ni de champs post-résolution (pas de fuite de label).
- Niveau de bruit ajusté pour une performance plafond de ~97–98% avec des modèles optimisés.

## 3- 📖 Interprétation de la Description des Données

#### * Nombre de lignes et de colonnes

In [3]:
# Nombre de lignes et colonnes
print("Nombre de lignes :", df.shape[0])
print("Nombre de colonnes :", df.shape[1])

Nombre de lignes : 50000
Nombre de colonnes : 33


#### * Types de variables

In [8]:
%pip install ace_tools
import ace_tools as tools
import pandas as pd

df = pd.read_csv("Support_tickets.csv")
# Identifier les types de variables
types_variables = df.dtypes.value_counts()
# Détail des colonnes avec leur type
colonnes_types = df.dtypes.reset_index()
colonnes_types.columns = ["Colonne", "Type"]
tools.display_dataframe_to_user("Types de variables", colonnes_types)

Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'ace_tools'

# III - 🧹 Data Preparation

## 1 - 🔎 Objectifs

La phase de **Data Preparation** consiste à transformer les données brutes en un format exploitable par les algorithmes de Machine Learning.  
Elle comprend :  
- le **nettoyage** des données,  
- la **gestion des valeurs manquantes**,  
- l’**encodage des variables catégorielles**,  
- la **normalisation/standardisation**,  
- la création de nouvelles **features** (feature engineering)